<a href="https://colab.research.google.com/github/abbasihsn/abbasihsn-privacy/blob/main/train_model_selected_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# copy data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q /content/drive/MyDrive/object_detection_datasets/ready_dataset_4classes_2000_train.zip -d /content/

In [ ]:

!nvidia-smi

Thu Sep 12 20:50:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.9/871.9 kB 53.4 MB/s eta 0:00:00


# prepare data

In [ ]:
from pathlib import Path
# Create a YAML configuration file for YOLO training
yaml_content = """
train: /content/ready_dataset/images/train
val: /content/ready_dataset/images/val
nc: 4
names: ['pedestrian', 'PassengerCar', 'LargeVehicle', 'RidableVehicle']
"""

yaml_file_path = Path('/content/dataset.yaml')
with open(yaml_file_path, 'w') as yaml_file:
    yaml_file.write(yaml_content)


In [ ]:
from pathlib import Path
import yaml
import os

# Define the path to the YAML file
yaml_file_path = Path('/content/dataset.yaml')

# Read the YAML file
with open(yaml_file_path, 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)

# Get the paths for the train and val directories from the YAML content
train_dir = Path(config['train'])
val_dir = Path(config['val'])

# Check if the train directory exists and contains files
if train_dir.exists() and any(train_dir.iterdir()):
    print(f"Train directory exists and contains files: {train_dir}")
    files = os.listdir(train_dir)
    print(f"Number of files in train directory: {len(files)}")
else:
    print(f"Train directory is either empty or does not exist: {train_dir}")

# Check if the val directory exists and contains files
if val_dir.exists() and any(val_dir.iterdir()):
    print(f"Val directory exists and contains files: {val_dir}")
    files = os.listdir(val_dir)
    print(f"Number of files in val directory: {len(files)}")
else:
    print(f"Val directory is either empty or does not exist: {val_dir}")


Train directory exists and contains files: /content/ready_dataset/images/train
Number of files in train directory: 17859
Val directory exists and contains files: /content/ready_dataset/images/val
Number of files in val directory: 1983


# Load the pretrained model

In [ ]:
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/object_detection_datasets/trained_models/best_4_selected_classes_55_epochs.pt")

In [ ]:
# Print the model architecture
print(model.model)

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 80, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(400, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
    

In [ ]:
all_params = list(model.model.parameters())

# Determine the number of layers to freeze
num_layers_to_freeze = len(all_params) - 10

In [ ]:
# Freeze the parameters of all layers except the last 5
for param in all_params[:num_layers_to_freeze]:
    param.requires_grad = False

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
model.train(data="/content/dataset.yaml", epochs=100, imgsz=640, batch = 8)

Ultralytics YOLOv8.2.92 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/object_detection_datasets/trained_models/best_4_selected_classes_55_epochs.pt, data=/content/dataset.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save

100%|██████████| 755k/755k [00:00<00:00, 37.0MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1    213120  ultralytics.nn.modules.block.SCDown          [320, 640, 3, 2]              
  6                  -1  6   4604160  ultralytics.nn.modules.block.C2fCIB          [640, 640, 6, True]           
  7                  -1  1    417920  ultralytics.nn.modules.block.SCDown          [640

100%|██████████| 6.25M/6.25M [00:00<00:00, 257MB/s]


AMP: checks passed ✅


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     57/100       9.7G      2.313      1.188      1.882        101        640: 100%|██████████| 2232/2232 [24:31<00:00,  1.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 124/124 [00:40<00:00,  3.08it/s]

                   all       1983      20515      0.749      0.581      0.651      0.413



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     58/100      9.74G      2.329       1.19      1.886        109        640:  26%|██▌       | 585/2232 [06:26<18:56,  1.45it/s]

In [ ]:
!nvcc --version